# Lane Detection on Video
This notebook demonstrates a lane detection pipeline on video using OpenCV. It processes each frame to identify and highlight lane lines, and outputs a processed video with detected lanes overlaid.

In [ ]:
import numpy as np
import matplotlib
import cv2
import imutils

## Moving Average Lists
Lists to store points for calculating the moving average of detected lane lines, which helps smooth the lane detection results over time.

In [ ]:
#list for moving average
ptsMAR = []
ptsMAL = []

## Snip Image Function
Defines a function to crop the region of interest from each video frame, focusing on the area where lanes are expected to appear.

In [ ]:
def snip_image(image):
	snip = image[(image.shape[0]-300):(image.shape[0]-50),(0):(image.shape[1])]
	return snip

## Mask Image Function
Defines a function to create a polygonal mask over the region of interest, isolating the lane area for further processing.

In [ ]:
def mask_image(image):
	mask = np.zeros((image.shape[0],image.shape[1]), dtype = "uint8")
	#points were changed to create a mask better suited to the new snip
	pts = np.array([[105,image.shape[0]-10],[105,image.shape[0]-25],[int(image.shape[1]/2)-20,150], 
		[260,150],[image.shape[1]-120, image.shape[0]-40],[image.shape[1]-120, image.shape[0]-10]],dtype=np.int32)
	
	#for point in pts:
		#cv2.circle(image, tuple(point), 4, (0,255,0), -1)
	cv2.fillConvexPoly(mask, pts,255)
	
	masked = cv2.bitwise_and(image,image,mask=mask)
	return masked


## Threshold Image Function
Defines a function to apply color thresholding to the masked image, highlighting lane lines based on color and brightness.

In [ ]:
def thresh_image(image):
	hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

	gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
	#added a new mask to pick up yellow colors
	yellowLower = np.array([0,65,164])
	yellowUpper = np.array([255,255,255])
	
	redLower = np.array([140,20,0])
	redUpper = np.array([230,100,50])
	
	#changed upper white to fully white
	whiteLower = np.array([30,0,65])
	whiteUpper = np.array([255,255,255])
	yellow_mask = cv2.inRange(hsv, yellowLower, yellowUpper)
	white_mask = cv2.inRange(hsv, whiteLower, whiteUpper)
	
	red_mask = cv2.inRange(hsv, redLower, redUpper)
	
	
	#creates a mask usng the yellow and white mask against each other 
	full_mask = cv2.bitwise_or(yellow_mask, white_mask)
	
	
	#creates a mask using the gray scale against the full mask 
	frame = cv2.bitwise_or(gray, full_mask)
	threshold = 100 
	black_white = cv2.threshold(frame, threshold, 255, cv2.THRESH_BINARY)[1]
	return black_white

## Blur Image Function
Defines a function to blur the thresholded image, reducing noise and improving edge detection.

In [ ]:
def blur_image(image):
	#blurs the image to help with edge detection
	return cv2.GaussianBlur(image, (21,21),0)

## Edge Detection Function
Defines a function to detect edges in the blurred image using the Canny edge detector, which helps identify lane boundaries.

In [ ]:

def edge_image(image):
	#identifys edges in the image
	return cv2.Canny(image,30,130)

## Hough Line Detection Function
Defines a function to detect lines in the edge-detected image using the Hough Transform, which is used to find lane lines.

In [ ]:
def lined_image(image):
	
	return cv2.HoughLines(image, 1, np.pi/180,30)


## Show Line Function
Defines a function to process detected lines, classify them as left or right lanes, and prepare them for averaging and visualization.

In [ ]:
def show_line(lines,snip,color):
	
	if lines is not None:
		#varibale for number of iteration for the lines of the left or right lane
		L = 0
		R = 0
		#placeholder varible for the points
		pt1L = (0,0)
		pt2L = (0,0)
		pt1R = (0,0)
		pt2R = (0,0)
		for i in range(0,len(lines)):
			rho = lines[i][0][0]
			theta = lines[i][0][1]
			
			a = np.cos(theta)
			b = np.sin(theta)
			
			x0 = a*rho
			#fixes offest between my mask and my original image 
			y0 = b*rho + 60
			
			pt1 = int(x0 + 1000*(-b)), int(y0+1000*(a))
			pt2 = int(x0 - 1000*(-b)), int(y0-1000*(a))
			
			#Checks if the x coordinate of the point is in the left hand side or right side of the screen by checking if its negative, 
			#this would mean its in the left hand side	
			if pt2[1] < 0:
				pt1L = tuple(map(sum,zip(pt1L,pt1)))
				pt2L = tuple(map(sum,zip(pt2L,pt2)))
				L +=1
			else:
				pt1R = tuple(map(sum,zip(pt1R,pt1)))
				pt2R = tuple(map(sum,zip(pt2R,pt2)))
				R +=1
			
		
		#checks if the the line returns none and makes the points of the line zero so they can be ignored later in the code 
		#or will take the points and find the average of the left and right points of the line
		if L == 0 and R == 0:
		
			pt1L = 0
			pt2L = 0
			pt1R = 0
			pt2R = 0

		elif L == 0 and R != 0:

			pt1L = 0
			pt2L = 0
			pt1R = tuple(map(lambda x: int(x/R), pt1R))
			pt2R = tuple(map(lambda x: int(x/R), pt2R))
		elif R == 0 and L != 0:
		
			pt1R = 0
			pt2R = 0
			pt1L = tuple(map(lambda x: int(x/L), pt1L))
			pt2L = tuple(map(lambda x: int(x/L), pt2L))
		else:
			pt1R = tuple(map(lambda x: int(x/R), pt1R))
			pt2R = tuple(map(lambda x: int(x/R), pt2R))
			pt1L = tuple(map(lambda x: int(x/L), pt1L))
			pt2L = tuple(map(lambda x: int(x/L), pt2L))
		#moves points to the moving average function
		moving_average(snip,(pt1L,pt2L),(pt1R,pt2R))

		
	
	return None

## Moving Average Function
Defines a function to compute the moving average of detected lane line points, draw the averaged lines, and overlay them on the video frame for smoother visualization.

In [ ]:
#used to get the moving average of the left and right lane lines points
def moving_average(snip,ptsL, ptsR):
	
	#list shown at the beginning which holds the points of the moving average 
	global ptsMAR
	global ptsMAL

	#if the points were from a none or were not picked up by the algorithm it will not add them to the list
	if ptsR != (0,0):

		ptsMAR.append(ptsR)

	if ptsL != (0,0):

		ptsMAL.append(ptsL)
	#if the list is greater than 50 it will get rid of the first element in the list
	if len(ptsMAR) > 50:

		ptsMAR.pop(0)

	if len(ptsMAL) > 50:

		ptsMAL.pop(0)
	
	#does the average og the points 
	pts1L = tuple(map(lambda x: int(x/len(ptsMAL)), tuple([sum(i) for i in zip(*[a_tuple[0] for a_tuple in ptsMAL])])))	
	pts2L = tuple(map(lambda x: int(x/len(ptsMAL)), tuple([sum(i) for i in zip(*[a_tuple[1] for a_tuple in ptsMAL])])))
	pts1R = tuple(map(lambda x: int(x/len(ptsMAR)), tuple([sum(i) for i in zip(*[a_tuple[0] for a_tuple in ptsMAR])])))
	pts2R = tuple(map(lambda x: int(x/len(ptsMAR)), tuple([sum(i) for i in zip(*[a_tuple[1] for a_tuple in ptsMAR])])))
	
	
	#using the average of points it creates a slope and y intercept to find the new points x coordinate for the wanted y coordinates based off of the images shape.
	#as you can see point pt1 will represent the bottom point on the screen since its at 480
	slopeL = (pts2L[1]-pts1L[1])/(pts2L[0]-pts1L[0])
	bL = pts1L[1]-pts1L[0]*slopeL
		
	pt1L = (int((480-bL)/slopeL),480)
	pt2L = (int((230-bL)/slopeL),230)

	
	
	slopeR = (pts2R[1]-pts1R[1])/(pts2R[0]-pts1R[0])
	bR = pts2R[1]-pts2R[0]*slopeR
	pt1R = (int((480-bR)/slopeR),480)
	pt2R = (int((230-bR)/slopeR),230)
	
	#creates a copy of the image to overlay the lines on 
	overlay = snip.copy()
	#creates a line based on the points created by the slope and inter
	cv2.line(overlay, pt2L,pt1L, (255,0,0), 10)
	cv2.line(overlay, pt1R,pt2R, (0,0,255), 10)
	
	#creates a transparency to the lines that were just created
	image = cv2.addWeighted(overlay, .4, snip, 1 - .4,0)
	
	cv2.imshow("lined", image)
			
			
	cv2.waitKey(1)

	return None

## Main Processing Loop
Defines the main function that loads the video, processes each frame through the lane detection pipeline, and writes the output video with detected lanes.

In [ ]:
def main():
	
	#loads the video
	vid = cv2.VideoCapture("video/test_video_01.mp4")
	width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
	height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
	fps = vid.get(cv2.CAP_PROP_FPS)
    # Define the codec and create VideoWriter object
	fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
	out = cv2.VideoWriter('video/output_lanes.mp4', fourcc, fps, (width, height))

	#will continue running till the video is over
	while (vid.isOpened()):
		# ret is equal to true while the video is running and image is the frame captured from the video
		ret, image = vid.read()
		#if video is running ths will run
		if ret == True:
		   
			
			snip = snip_image(image)
			
			mask = mask_image(snip)

			frame = thresh_image(mask)

			blur = blur_image(frame)

			edged= edge_image(blur)
		
			lines = lined_image(edged)
			
			
			show_line(lines,image,(0,0,255))
			
			out.write(image)
			#if the video is running and q is pressed on the keyboard the while loop will break and video will end
			if cv2.waitKey(1) & 0xFF == ord('q'):
				break
		
		#if the the video ends by itself the video will be closed and all windows will be destroyed, edning the code
		if ret == False:
			vid.release()
			cv2.destroyAllWindows()
	

## Run the Lane Detection Pipeline
Executes the main function to start processing the video and display the results.

In [ ]:
main()